# Block 1: Medizininformatik

Eure Ansprechpartner sind Hannes Ulrich und Joshua Wiedekopf.

In diesem Block geht es darum, die Daten aus der TCGA-Studie aus disjunkten Quellen in ein Format zu bringen, mit dem in den folgenden Blöcken weiter gearbeitet werden kann.

Hierzu werden wir auf einen FHIR-Server zugreifen, um die Patientendaten abzurufen, während die klinischen Daten im CSV-Format vorliegen.

## Pakete laden und installieren

**Dieser Block muss einmal unbedingt ausgeführt werden, damit in Eurer Runtime die notwendigen Pakete zur Verfügung stehen!**

In [ ]:
#@title Pakete installieren
install.packages("fhircrackr")
install.packages("assertthat")
install.packages("assertr")

Sollten Konflikte gemeldet werden, ist dies in der Regel in Ordnung.

In [ ]:
#@title Pakete laden
library(assertthat)
library(fhircrackr)
library(tidyverse)
library(glue)
library(assertr)

In [41]:
#@title Hilfsfunktionen
fill_me <- function(name, description="") {
  error_msg = glue("Die Variable/der Parameter '{name}' ist nicht gefüllt: {description}")
  stop(error_msg)
}
fun_me <- function(description) {
  error_msg = glue("Hier fehlt ein Funktionsaufruf: {description}")
  stop(error_msg)
}
test <- function(count=1, fun) {
  fun()
  print(ifelse(count == 1, "1 Test erfolgreich", str_glue("{count} Tests erfolgreich")))
}


## Daten laden vom FHIR-Server

Habt ihr Probleme bei der Umsetzung dieser Aufgabe, ist die [Dokumentation von HL7 FHIR 🇬🇧](https://hl7.org/fhir/R5/) hilfreich. Insbesondere die folgenden Seiten können wichtig sein:
- [FHIR Search](https://hl7.org/fhir/R5/search.html)
- [Ressource *Patient*](https://hl7.org/fhir/R5/patient.html)
- [Ressource *Condition*](https://hl7.org/fhir/R5/condition.html)
- [Ressource *Observation*](https://hl7.org/fhir/R5/observation.html)
- [Ressource *ResearchStudy*](https://hl7.org/fhir/R5/researchstudy.html)
- [Ressource *ResearchSubject*](https://hl7.org/fhir/R5/researchsubject.html)

Darüber hinaus ist die [Dokumentation vom `fhircrackr`-R-Paket 🇬🇧](https://cran.r-project.org/web/packages/fhircrackr) sehr hilfreich:
- [Download FHIR resources](https://cran.r-project.org/web/packages/fhircrackr/vignettes/downloadResources.html)


In [ ]:
#@title Endpunkt-Variable festlegen
endpoint="https://fhir.imi.uni-luebeck.de/fhir/"
# die Adresse unseres FHIR-Servers. Unter https://fhir.imi.uni-luebeck.de ist auch eine einfache GUI verfügbar.

## Aufgabe 1

Ladet alle Ressourcen vom Typ `ResearchStudy` mit folgendem Identifier:
- `system` = `https://www.cbioportal.org/study`
- `value`  = `paad_tcga_pan_can_atlas_2018`

Dabei sollte genau eine Ressource gefunden werden.

In [ ]:
#@title A1: Erste Anfrage nach der ResearchStudy
request <- fhir_url(
  url = endpoint,
  resource = fill_me("resource", "Hier soll der ResourceType, den wir suchen, rein"),
  parameters = list(
    fill_me("parameters", "Hier können Suchparameter angegeben werden.")
  )
)
studies_bundle <- fun_me("Die Suchanfrage muss noch an den Server geschickt werden")
str(studies_bundle) # Gibt die Datenstruktur aus, mit der wir arbeiten
head(studies_bundle) # gibt die ersten paar Zeilen aus

In [ ]:
#@title Lösung von Aufgabe 1
request <- fhir_url(
  url = endpoint,
  resource = "ResearchStudy",
  parameters = list(
    "identifier" = "https://www.cbioportal.org/study|paad_tcga_pan_can_atlas_2018"
  )
)
studies_bundle <- fhir_search(request, verbose=0)
str(studies_bundle)
head(studies_bundle)

In [ ]:
#@title Test der Lösung
test(count=2, fun=function() {
  assert_that(length(studies_bundle) == 1)
  assert_that(is(studies_bundle, "fhir_bundle_list"))
})

## Aufgabe 2

Gut, jetzt haben wir ein FHIR-Bundle. Damit wir innerhalb von R damit auch arbeiten können, verwenden wir die `fhir_crack`-Funktion von `fhircrackr`, um die Daten in die R-native Datenstruktur `dataframe` zu bekommen.

Das "Flachklopfen" der FHIR-Ressourcen ist in der Dokumentation von `fhircrackr` [beschrieben](https://cran.r-project.org/web/packages/fhircrackr/vignettes/fhircrackr_intro.html#flattening-resources).

Es muss also eine Tabellen-Definition erstellt werden und dann diese Definition auf unser Bundle angewendet werden. Uns interessiert nicht alles, sogar fast das meiste nicht. Daher können wir die Spaltendefinitionen relativ knapp fassen. Damit FHIR-Referenzen aufgelöst werden können, benötigen wir die technische `id` der ResearchStudy. Damit wir selber mit dem DataFrame etwas anfangen können, geben wir noch den `name` und die Komponenten des `identifier` aus.

**Achtung Verwechslungsgefahr**: In FHIR gibt es häufig sowohl eine technische ID (meist im Parameter `id`), unter der die Ressource auf einem Server verfügbar ist, und mittels derer Ressourcen verlinkt werden, als auch eine oder mehrere logische IDs, die auf Basis bestimmter Regeln, teilweise auch durch Externe, vergeben werden. Beispiel für Patienten: Die FHIR-Ressource für die Patientin [`Grace B. Hopper`](https://en.wikipedia.org/wiki/Grace_Hopper) ist auf einem FHIR-Server unter der ID `facade00-0000-4000-a000-000000000000` verfügbar, auf einem anderen Server aber unter der ID `c0c0a000-0000-4000-a000-000000000000`. Beide identifieren die Person in der Anwendungssoftware aufgrund ihres logischen Identifiers:

```json
{
  "resourceType": "Patient",
  "id": "facade00-0000-4000-a000-000000000000"
  "identifier": [
    {
      "system": "http://iau.org/working-group-for-small-bodies-nomenclature",
      "value: "5773 Hopper",
      "type": {
        "text": "A minor planet has been named after this person"
      }
    }
  ],
  "name": [
    {
      "given": ["Grace", "Brewster"],
      "family": "Hopper",
      "use": "official"
    }
  ]
}
```

Es ist möglich, in FHIR Referenzen aufgrund logischer IDs zu vergebenen, die meisten FHIR-Server lösen diese aber nicht zur Laufzeit auf. Daher wird die technische ID zur Referenzierung verwendet.

### Zielschema

#### ResearchStudy

| id | name | identifier_system | identifier_value |
|---|---|---|---|
| chr \| num | chr | chr | chr |

In [ ]:
#@title A2: `fhir_crack` verwenden
table_description_studies <- fhir_table_description(
  resource=fill_me('resource', 'hier soll wieder der ResourceType rein'),
  cols=c(
    fill_me('Ausgabespalte', 'erste Spalte?')=fill_me('XPath-Ausdruck für die erste Spalte im FHIR-XML'),
    fill_me('Ausgabespalte', 'zweite Spalte?')=fill_me('XPath-Ausdruck für die zweite Spalte im FHIR-XML')#,
    #...
  )
)
studies <- fhir_crack(bundles=studies_bundle, design=table_description_studies, verbose=0)
studies
study_id = fun_me("In dieser Variable soll am Ende genau eine Study-ID (FHIR-ID) stehen.")

In [ ]:
#@title Lösung von Aufgabe 2
table_description_studies <- fhir_table_description(
  resource="ResearchStudy",
  cols=c(
    id="id",
    name="name",
    identifier_system="identifier/system",
    identifier_value="identifier/value"
  )
)
studies <- fhir_crack(bundles=studies_bundle, design=table_description_studies, verbose=0)
studies
study_id = (studies$id)[1]

In [ ]:
#@title Test der Lösung
test(count=3, fun=function() {
  studies |>
    verify(has_all_names("id", "name")) |>
    verify(identifier_value == 'paad_tcga_pan_can_atlas_2018')
  assert_that(nrow(studies) == 1)
})

## Aufgabe 3

Wir haben jetzt eine Referenz auf die ResearchStudy, sodass wir auf Basis dieser nach den inkludierten Patienten suchen können - andere Patienten wollen wir natürlich nicht abfragen, weil wir es nicht dürfen 😎.

Die Teilnahme eines Patienten ist an die Ressource `ResearchSubject` geknüpft, die Brücke zwischen `ResearchStudy` und `Patient`. In `ResearchSubject` wird eine studieninterne ID zugewiesen, die wir benötigen, in `Patient` ist neben `gender` und `deceased` auch die ID relevant, damit wir über diese die Prozedur- und Diagnose-Ressourcen im nächsten Schritt anfragen können. Später werden wir die interne Patienten-ID wieder aus dem Datensatz entfernen, damit uns keine direkte Reidentifikation der Patienten in unserer weiteren Forschung möglich ist.

Wir werden also die `ResearchSubject`-Ressourcen, die auf unsere `ResearchStudy` verweisen, anfragen. Danach fragen wir die Patienten an, die in `ResearchSubject` referenziert werden. Das ist aber umständlich, daher nutzen wir die [Möglichkeit des `include`-Parameters](https://www.hl7.org/fhir/search.html#_revinclude), um die referenzierten Patienten direkt mit auszugeben.

Dies wird auch [durch fhircrackr unterstützt](https://cran.r-project.org/web/packages/fhircrackr/vignettes/flattenResources.html#extracting-more-than-one-resource-type).

Zur Datenmanipulation sind Pipes `|>` und das Paket `dyplr` sehr hilfreich:

```r
data |>
  filter(x='foo') |>
  mutate(y=x+3)
```

Mehr Informationen dazu im [Data Wrangling Cheat Sheet](https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf) von `dplyr`.

---

Am Ende soll ein Data Frame mit den Daten aus dem `ResearchSubject` und den Daten aus dem `Patient`. Wir müssen die Daten also joinen:

![Join-Typen](https://www.ionos.at/digitalguide/fileadmin/DigitalGuide/Screenshots/DE-SQL-Join-Typen.png)

*Bildquelle: IONOS, https://www.ionos.at/digitalguide/hosting/hosting-technik/sql-join/*

---

### Zielschemata:

#### ResearchSubject

| subject_id | patient_id | subject_identifier_system | subject_identifier_value |
|---|---|---|---|
| chr \| num | chr \| num | chr | chr |

#### Patient

| patient_id | deceasedBoolean | gender |
|---|---|---|
| chr \| num | logi | chr |

*Beachte: Den Identifier des Patienten sollten wir lieber nicht anfragen!*

In [ ]:
#@title A3: ResearchSubject und Patient anfragen und joinen
subject_patient_url <- fhir_url(
  url = endpoint,
  resource = fill_me("ResourceType"),
  parameters = list(
    "study"=fill_me("Filter auf Studie", "Wir wollen nur ResearchSubject für unsere Studie"),
    "_include"=fill_me("Include", "Wir wollen die referenzierten Patienten mit in der Ausgabe")
  )
)

subject_bundle <- fhir_search(subject_patient_url, verbose=0)
res_subject_design <- fhir_table_description(
  resource = fill_me("ResourceType"),
  cols=fill_me("cols", "Mapping für das erste Design")
)
pat_design <- fhir_table_description(
  resource = fill_me("ResourceType"),
  cols=fill_me("cols", "Mapping für das zweite Design")
)

combined_design <- fhir_design(ResearchSubject=res_subject_design, Patient=pat_design)
data <- fhir_crack(bundles=subject_bundle, design=combined_design, verbose=0)

# Reduktion auf die nötigsten Daten und leichte Manipulationen in einer Spalte (um einen einfacheren Join zu machen)
research_subject_data <- data$ResearchSubject |>
  mutate(patient_id=fun_me("Präfix 'Patient/' in der Spalte patient_id entfernen"))
#Datentypanpassung in einer Spalte
patient_data <- data$Patient |>
  mutate(deceasedBoolean=fun_me("Datentyp der Spalte deceasedBoolean soll logi sein, nicht chr"))

joined_subject_data <- research_subject_data |>
  fun_me('join-typ', by=fill_me('Join-bedingung'))
head(joined_subject_data)

In [ ]:
#@title Lösung von Aufgabe 3
subject_patient_url <- fhir_url(
  url = endpoint,
  resource = "ResearchSubject",
  parameters = list(
    "study"=glue("ResearchStudy/{study_id}"),
    "_include"="ResearchSubject:patient"
  )
)

subject_bundle <- fhir_search(subject_patient_url, verbose=0)
res_subject_design <- fhir_table_description(
  resource = "ResearchSubject",
  cols=c(
    subject_id="id",
    patient_id="subject/reference",
    subject_identifier_system="identifier/system",
    subject_identifier_value="identifier/value"
  )
)
pat_design <- fhir_table_description(
  resource = "Patient",
  cols=c(
    patient_id="id",
    deceasedBoolean="deceasedBoolean",
    gender="gender"
  )
)

combined_design <- fhir_design(ResearchSubject=res_subject_design, Patient=pat_design)
data <- fhir_crack(bundles=subject_bundle, design=combined_design, verbose=0)

# Reduktion auf die nötigsten Daten und leichte Manipulationen in einer Spalte (um einen einfacheren Join zu machen)
research_subject_data <- data$ResearchSubject |>
  mutate(patient_id=gsub("Patient/", "", patient_id))
#Datentypanpassung in einer Spalte
patient_data <- data$Patient |>
  mutate(deceasedBoolean=as.logical(deceasedBoolean))

str(patient_data)

joined_subject_data <- research_subject_data |>
  left_join(patient_data, by="patient_id")
head(joined_subject_data)

In [ ]:
#@title Test der Lösung
test(count=4, fun=function() {
  joined_subject_data |>
    verify(has_all_names("subject_id", "patient_id", "subject_identifier_value", "subject_identifier_system", "deceasedBoolean", "gender")) |>
    verify(any(patient_id == 12)) |>
    verify(any(subject_id == 20))
  assert_that(nrow(joined_subject_data) > 20)
})

## Aufgabe 4



Nun wollen wir die mit den Patienten verknüpften Conditions und Procedures anfragen. Letztendlich ist hier viel ähnliches zu der vorherigen Aufgabe dabei.

Am Ende dieser Aufgabe haben wir eine große Tabelle, bei der alle im FHIR-Repository vorliegenden Daten in den Spalten auftauchen, die uns aber keine direkten Verweise zu den Ressourcen mehr gibt. Lediglich das Studien-Pseudonym darf noch die Person identifizieren.

---

### Zielschemata

#### Condition

| condition_id | patient_id | condition_code | condition_display | condition_onsetAge_unit | condition_onsetAge_value |
|---|---|---|---|---|---|
| chr \| num | chr \| num | chr | chr | chr | num |

#### Procedure

| procedure_id | patient_id | procedure_code | procedure_display | procedure_system |
|---|---|---|---|---|
| chr \| num | chr \| num | chr | chr | chr |

#### Join-Schema (Spalten sind Zeilen...)

| Spalte | Datentyp |
|---|---|
| subject_identifier_system | chr |
| subject_identifier_value | chr |
| deceasedBoolean | logi |
| gender | chr \| factor |
| condition_code | chr |
| condition_display | chr |
| condition_onsetAge_unit | chr |
| condition_onsetAge_value | num |
| procedure_code | chr |
| procedure_display | chr |
| procedure_system | chr |

In [ ]:
#@title A4: Condition und Procedure anfragen
patient_id_join <- fun_me("Alle Patienten-IDs zusammenfügen, damit nur deren Daten angefragt werden")
condition_url <- fun_me("FHIR-Url für die Suche nach Conditions mit Beschränkung auf das Subject")
condition_data <- fun_me("direkt die URL anfragen und cracken") |>
  fun_me("notwendige Modifikationen der Ausgabe durchführen")
procedure_url <- fun_me("FHIR-Url für die Suche nach Conditions mit Beschränkung auf das Subject")
procedure_data <- fun_me("direkt die URL anfragen und cracken") |>
  fun_me("notwendige Modifikationen der Ausgabe durchführen")

joined_clinical_data <- joined_subject_data |>
  fun_me("FHIR-Tabelle fertig stellen gemäß Zielschema")
head(joined_clinical_data)

In [ ]:
#@title Lösung von Aufgabe 4
patient_id_join <- paste(joined_subject_data$patient_id, collapse = ',')
condition_url <- fhir_url(
  url=endpoint,
  resource="Condition",
  parameters=c(
    "subject"=patient_id_join
  )
)
condition_data <- fhir_crack(
  bundles=fhir_search(condition_url, verbose=0),
  design=fhir_table_description(
    resource="Condition",
    cols=c(
      condition_id="id",
      patient_id="subject/reference",
      condition_code="code/coding/code",
      condition_display="code/coding/display",
      condition_onsetAge_unit="onsetAge/unit",
      condition_onsetAge_value="onsetAge/value"
    )
  )
) |>
  mutate(patient_id = gsub("Patient/", "", patient_id))

procedure_url <- fhir_url(
  url=endpoint,
  resource="Procedure",
  parameters=c(
    subject=patient_id_join
  )
)
procedure_data <- fhir_crack(
  bundles=fhir_search(procedure_url, verbose=0),
  design=fhir_table_description(
    resource="Procedure",
    cols=c(
      "procedure_id"="id",
      "patient_id"="subject/reference",
      "procedure_code"="code/coding/code",
      "procedure_display"="code/coding/display",
      "procedure_system"="code/coding/system"
    )
  )
) |>
  mutate(patient_id = gsub("Patient/", "", patient_id))

joined_clinical_data <- joined_subject_data |>
  left_join(condition_data, by="patient_id") |>
  left_join(procedure_data, by="patient_id") |>
  mutate(condition_onsetAge_value=as.numeric(condition_onsetAge_value)) |>
  select(-subject_id, -patient_id, -condition_id, -procedure_id)
head(joined_clinical_data)

In [ ]:
#@title Test der Lösung
test(count=4, fun=function() {
  joined_clinical_data |>
    verify(has_all_names("subject_identifier_value", "subject_identifier_system", "deceasedBoolean", "gender")) |>
    verify(has_all_names("condition_code", "condition_display", "condition_onsetAge_unit", "condition_onsetAge_value")) |>
    verify(has_all_names("procedure_code", "procedure_display", "procedure_system"))

  assert_that(nrow(joined_clinical_data) > 20)
})

## Aufgabe 5

Jetzt laden wir den Datensatz mit weiteren klinischen Daten. Dieser ist im CSV-Format unter https://drive.google.com/file/d/1Vs7ZMva965Q_YPVcTxG5P2qmInuuyaIM/view?usp=sharing herunterzuladen und oben links in den Session Storage laden:

![Session Storage](https://f003.backblazeb2.com/file/script-junk/gmds-2024-summit/load-csv-colab.png)

Ein vollständiges Zielschema ist zu lang, lediglich eine Änderung ist unten im Code vorgeschlagen.

In [ ]:
#@title A5: CSV-Laden und joinen
csv_data <- fun_me("Daten laden")

all_data <- csv_data |>
  fun_me("Die Spalte patient_id hat einen verwirrenden Namen, sie sollte lieber subject_identifier sein") |>
  fun_me("Wieder ein Join")

head(all_data)
str(all_data)

In [ ]:
#@title Lösung für Aufgabe 5
csv_data <- read.csv("data.csv", sep=",")

all_data <- csv_data |>
  rename(subject_identifier=patient_id) |>
  left_join(joined_clinical_data, by=join_by(subject_identifier == subject_identifier_value))
head(all_data)
str(all_data)

## Aufgabe 6

Unser Datensatz ist jetzt soweit fertig. Zum Abschluss nutzen wir die umfassenden Möglichkeiten von R für ein paar erste Gehversuche in der Datenanalyse. Hierzu nutzen wir das Paket `ggplot` aus dem Tidyverse [[Docs]](https://ggplot2.tidyverse.org/index.html).

Wie bei `dplyr` ist das Cheat Sheet von `ggplot` sehr hilfreich: [[Cheat Sheet]](https://github.com/rstudio/cheatsheets/blob/master/data-visualization.pdf). Zum Testen der Plot-Funktionen kann auch der kleine Block hier verwendet werden, um mit einem einfachen Datensatz zu spielen.

In `ggplot` werden Daten auf *aesthetics* gemapped, die dann durch *layer* graphisch dargestellt werden. In dieser Aufgabe gibt es weniger Hilfestellung direkt in der Aufgabe, probiert gerne einfach mit den Daten rum, was sich direkt darstellen lässt.

In [ ]:
#@title `ggplot`-Beispiel
# von: https://nbisweden.github.io/workshop-r/2011/lab_ggplot2.html
# verwendet einen mit R mitgelieferten Datensatz, es geht um Pflanzen, deren Blattform analysisert wurde
data(iris)
str(iris) # Species ist ein `Factor`, das ist eine kompakte Repräsentation von kategorischen Variablen, bei denen die Strings durch Codes ersetzt werden, also setosa=1, etc.
ggplot(data=iris,mapping=aes(x=Petal.Length,y=Petal.Width,color=Species))+
  geom_point() +
  geom_smooth(formula=y~x, method="lm")

Die beiden folgenden Grafiken werden durch die Musterlösung generiert, die ihr reproduzieren könnt:

|Alter bei Diagnose pro Geschlecht|Survival pro ICD-10-Code|
|-|-|
|![Grafik 1](https://f003.backblazeb2.com/file/script-junk/gmds-2024-summit/ggplot1.png)|![Grafik 2](https://f003.backblazeb2.com/file/script-junk/gmds-2024-summit/ggplot2.png)|

In [ ]:
#@title A6: `ggplot`
ggplot(data=all_data, aes=fill_me("Sinnvolles Aesthetic-Mapping")) +
  fill_me("Sinnvolles Geom") +
  fill_me("Theming, Beschriftung, ....")

data |> # Pipes gehen hier auch :)
  ggplot(aes=fill_me("Anderes aes")) +
  fill_me("anderes sinnvolles Geom") +
  fill_me("Theming, Beschriftung, ...")

In [ ]:
#@title Lösung für Aufgabe 6 (erster Plot)
ggplot(data=all_data, aes(x=condition_onsetAge_value, col=gender)) +
  geom_histogram(aes(y = after_stat(density), col=gender), fill='white', bins=30) +
  geom_line(stat="density", linewidth = 1.5) +
  labs(title="Age at diagnoses, distribution per gender", x="Age at diagnosis", y="Density", col="Gender")

In [ ]:
#@title Lösung für Aufgabe 6 (zweiter Plot)
ggplot(data=all_data, aes(x=condition_code, col=disease.specific_survival_status, fill=disease.specific_survival_status)) +
  geom_bar(stat='count') +
  labs(title="Disease-specific survival per ICD-10 code", x="ICD-10 code", y="Count", col="Disease-specific survival", fill="Disease-specific survival")